# Road-Sign-Detection
## Problem statement
This dataset contains 877 images of 4 distinct classes for the objective of road sign detection.
Bounding box annotations are provided in the PASCAL VOC format
The classes are:Trafic Light;Stop;Speedlimit;Crosswalk.
## To Predict
Road Sign Detection using Convolutional Neural Networks

## Step 0: Import all the necessary Libraries

In [51]:
import os
import numpy as np
import cv2
import xml.etree.ElementTree as ET
from sklearn.model_selection import train_test_split
import pandas as pd
from tensorflow import keras
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical

## Step 1: Data Preparation

In [52]:
image_folder = 'C:\\Users\\Vishnu\\Downloads\\archive\\images'
annotation_folder = 'C:\\Users\\Vishnu\\Downloads\\archive\\annotations'

In [53]:
image_filenames = os.listdir(image_folder)
annotations_filenames = os.listdir(annotation_folder)

These lines list all the filenames in the image and annotation folders using os.listdir().

In [54]:
# Create empty lists to store the data
images = []
labels = []

## Step 2: Data Preprocessing

In [55]:
# Loop through each image
for image_filename in image_filenames:
    # Load the image
    image = cv2.imread(os.path.join(image_folder, image_filename))
    
    # Define the desired width and height for resizing
    desired_width = 224
    desired_height = 224
    
    # Resize the image to a consistent shape
    image = cv2.resize(image, (desired_width, desired_height))
    
    images.append(image)

    # Parse the corresponding XML annotation
    annotation_filename = image_filename.replace('.png', '.xml')
    annotation_path = os.path.join(annotation_folder, annotation_filename)
    tree = ET.parse(annotation_path)
    root = tree.getroot()

    # Extract bounding box coordinates and class label
    xmin = int(root.find('.//xmin').text)
    ymin = int(root.find('.//ymin').text)
    xmax = int(root.find('.//xmax').text)
    ymax = int(root.find('.//ymax').text)
    class_label = root.find('.//name').text

    labels.append(class_label)

* This loop iterates through each image filename in the image_filenames list.
* It reads each image using cv2.imread() and resizes it to the desired width and height using cv2.resize().
* The resized image is appended to the images list.
* The corresponding XML annotation filename is obtained by replacing the extension of the image filename with '.xml'.
* The XML annotation file is parsed using xml.etree.ElementTree and the root element is obtained.
* The bounding box coordinates (xmin, ymin, xmax, ymax) and class label are extracted from the XML.
* The class label is appended to the labels list.

In [56]:
# Convert images and labels to numpy arrays
images = np.stack(images)
labels = np.array(labels)

In [57]:
# Map string labels to integer values
label_mapping = {label: i for i, label in enumerate(np.unique(labels))}
labels_mapped = np.array([label_mapping[label] for label in labels])

These lines create a label mapping dictionary that maps unique class labels to integer values.
The labels numpy array is mapped to its corresponding integer values using list comprehension.
The labels_mapped array contains the mapped integer labels.

In [58]:
# Perform one-hot encoding of the labels
num_classes = len(np.unique(labels_mapped))
labels_encoded = to_categorical(labels_mapped, num_classes)

The num_classes variable is set to the number of unique mapped labels.
The labels_mapped array is one-hot encoded using to_categorical() from tensorflow.keras.utils.

In [59]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels_encoded, test_size=0.2, random_state=42)

The dataset is split into training and testing sets using train_test_split() from sklearn.model_selection.
The images and labels_encoded arrays are split into X_train, X_test, y_train, and y_test with a test size of 20% and a random state of 42.

## Step 3: Model Architecture

In [60]:
# Define the CNN model architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(desired_width, desired_height, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

A sequential model is created using Sequential() from tensorflow.keras.models.
The model architecture consists of a 2D convolutional layer, max pooling layer, flatten layer, and two dense (fully connected) layers.
The first convolutional layer has 32 filters, a kernel size of (3, 3), and uses the ReLU activation function.
The max pooling layer has a pool size of (2, 2).
The flatten layer flattens the output from the previous layer.
The first dense layer has 128 units with the ReLU activation function.
The final dense layer has the number of units equal to the number of classes and uses the softmax activation function.

## Step 4: Model Training

In [61]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=32)

Epoch 1/10
22/22 [==============================] - 11s 411ms/step - loss: 4930.6401 - accuracy: 0.6120
Epoch 2/10
22/22 [==============================] - 9s 390ms/step - loss: 292.9170 - accuracy: 0.7161
Epoch 3/10
22/22 [==============================] - 9s 406ms/step - loss: 13.9364 - accuracy: 0.8859
Epoch 4/10
22/22 [==============================] - 9s 388ms/step - loss: 3.8224 - accuracy: 0.9230
Epoch 5/10
22/22 [==============================] - 9s 403ms/step - loss: 0.9695 - accuracy: 0.9857
Epoch 6/10
22/22 [==============================] - 9s 389ms/step - loss: 0.1706 - accuracy: 0.9943
Epoch 7/10
22/22 [==============================] - 9s 398ms/step - loss: 0.1941 - accuracy: 0.9957
Epoch 8/10
22/22 [==============================] - 9s 395ms/step - loss: 0.1543 - accuracy: 0.9872
Epoch 9/10
22/22 [==============================] - 9s 392ms/step - loss: 1.8212 - accuracy: 0.9558
Epoch 10/10
22/22 [==============================] - 9s 401ms/step - loss: 0.4914 - accuracy:

The model is compiled with the categorical cross-entropy loss function, the Adam optimizer, and accuracy as the metric.
The training data (X_train and y_train) is used to train the model using fit() function.
The training is performed for 10 epochs with a batch size of 32.

## Step 5: Prediction and Evaluation

In [62]:
model.evaluate(X_test,y_test)

6/6 [==============================] - 1s 93ms/step - loss: 36.9415 - accuracy: 0.7670


[36.941471099853516, 0.7670454382896423]

In [63]:
predictions = model.predict(X_test)
predicted_classes = np.argmax(predictions, axis=1)

6/6 [==============================] - 1s 93ms/step


In [64]:
# Convert predicted classes to original label format
predicted_labels = np.array([np.unique(labels)[pred_class] for pred_class in predicted_classes])

In [65]:
# Create a DataFrame with image filename, predicted class, and original class
results_df = pd.DataFrame({
    'Image Filename': image_filenames[:len(predicted_labels)],
    'Predicted Class': predicted_labels,
    'Original Class': np.array([np.unique(labels)[np.argmax(label)] for label in y_test])
})

In [66]:
# Save the DataFrame to a CSV file
results_df.to_csv('test_results.csv', index=False)

The model is used to make predictions on the testing data (X_test) using predict().
The predicted classes are obtained by finding the indices of the highest probability values using np.argmax().
The predicted classes are then converted back to their original label format using the label mapping dictionary.
A DataFrame (results_df) is created to store the image filename, predicted class, and original class for each test image.
The DataFrame is saved to a CSV file named 'test_results.csv' using to_csv().

In [67]:
r = pd.read_csv("test_results.csv")

In [68]:
r.head()

,Image Filename,Predicted Class,Original Class
0,road0.png,speedlimit,speedlimit
1,road1.png,trafficlight,stop
2,road10.png,speedlimit,crosswalk
3,road100.png,speedlimit,stop
4,road101.png,speedlimit,speedlimit


## Saving Model and Taking Input

In [69]:
model.save('C:\\Users\\Vishnu\\Downloads\\Road-sign')

INFO:tensorflow:Assets written to: C:\Users\Vishnu\Downloads\Road-sign\assets


INFO:tensorflow:Assets written to: C:\Users\Vishnu\Downloads\Road-sign\assets


Saving the trained model

In [88]:
loaded_model = load_model('C:\\Users\\Vishnu\\Downloads\\Road-sign')

Loading the trained model for prediction

In [84]:
input_image = cv2.imread('C:\\Users\\Vishnu\\Downloads\\archive\\images\\road0.png')
image_size = (224, 224)
preprocessed_image = cv2.resize(input_image, image_size) / 255.0
preprocessed_image = np.expand_dims(preprocessed_image, axis=0)

 Loading and preprocessing the input image for prediction

In [85]:
predictions = loaded_model.predict(preprocessed_image)
predicted_label_index = np.argmax(predictions[0])

1/1 [==============================] - 0s 68ms/step


Making predictions on the input image

In [89]:
class_labels = ['Traffic Light', 'Stop', 'Speed Limit', 'Crosswalk']
predicted_label = class_labels[predicted_label_index]

Mapping the predicted label index to the corresponding class label

### Print the predicted label

In [87]:
print('Predicted Label:', predicted_label)

Predicted Label: Crosswalk


## Accuracy of the Model: 0.7670454382896423